In [4]:
import xgt
import os

In [5]:
if os.environ.get('https_proxy'):
 del os.environ['https_proxy']
if os.environ.get('http_proxy'):
 del os.environ['http_proxy']

In [6]:
conn=xgt.Connection()
#conn.__version__
#conn.about()
#conn.server_version

XgtConnectionError: Failure on connection initialization.

In [ ]:
# Create a Vertex Frame named "devices"
try:
  devices = conn.get_vertex_frame('Devices')
except xgt.XgtNameError:
  devices = conn.create_vertex_frame(
              name='Devices',
              schema=[['device', xgt.TEXT]],
              key='device')
devices

In [ ]:
# Create an Edge Frame named "netflow"

try:
  netflow = conn.get_edge_frame('Netflow')
except xgt.XgtNameError:
  netflow = conn.create_edge_frame(
            name='Netflow',
            schema=[['epochtime', xgt.INT],
                    ['duration', xgt.INT],
                    ['srcDevice', xgt.TEXT],
                    ['dstDevice', xgt.TEXT],
                    ['protocol', xgt.INT],
                    ['srcPort', xgt.INT],
                    ['dstPort', xgt.INT],
                    ['srcPackets', xgt.INT],
                    ['dstPackets', xgt.INT],
                    ['srcBytes', xgt.INT],
                    ['dstBytes', xgt.INT]],
            source=devices,
            target=devices,
            source_key='srcDevice',
            target_key='dstDevice')
netflow

In [ ]:
try:
  events1v = conn.get_edge_frame('Events1v')
except xgt.XgtNameError:
  events1v = conn.create_edge_frame(
           name='Events1v',
           schema=[['epochtime', xgt.INT],
                   ['eventID', xgt.INT],
                   ['logHost', xgt.TEXT],
                   ['userName', xgt.TEXT],
                   ['domainName', xgt.TEXT],
                   ['logonID', xgt.INT],
                   ['processName', xgt.TEXT],
                   ['processID', xgt.INT],
                   ['parentProcessName', xgt.TEXT],
                   ['parentProcessID', xgt.INT]],
           source=devices,
           target=devices,
           source_key='logHost',
           target_key='logHost')
events1v

In [ ]:
try:
  events2v = conn.get_edge_frame('Events2v')
except xgt.XgtNameError:
  events2v = conn.create_edge_frame(
           name='Events2v',
           schema = [['epochtime',xgt.INT],
                     ['eventID',xgt.INT],
                     ['logHost',xgt.TEXT],
                     ['logonType',xgt.INT],
                     ['logonTypeDescription',xgt.TEXT],
                     ['userName',xgt.TEXT],
                     ['domainName',xgt.TEXT],
                     ['logonID',xgt.INT],
                     ['subjectUserName',xgt.TEXT],
                     ['subjectDomainName',xgt.TEXT],
                     ['subjectLogonID',xgt.TEXT],
                     ['status',xgt.TEXT],
                     ['src',xgt.TEXT],
                     ['serviceName',xgt.TEXT],
                     ['destination',xgt.TEXT],
                     ['authenticationPackage',xgt.TEXT],
                     ['failureReason',xgt.TEXT],
                     ['processName',xgt.TEXT],
                     ['processID',xgt.INT],
                     ['parentProcessName',xgt.TEXT],
                     ['parentProcessID',xgt.INT]],
            source = 'Devices',
            target = 'Devices',
            source_key = 'src',
            target_key = 'logHost')
events2v

In [ ]:
# Utility to print the sizes of data currently in xGT
def print_data_summary():
  print('Devices (vertices): {:,}'.format(devices.num_vertices))
  print('Netflow (edges): {:,}'.format(netflow.num_edges))
  print('Host event 1-vertex (edges): {:,}'.format(events1v.num_edges))
  print('Host event 2-vertex (edges): {:,}'.format(events2v.num_edges))
  print('Total (edges): {:,}'.format(
      netflow.num_edges + events1v.num_edges + events2v.num_edges))
    
print_data_summary()

In [7]:
%%time
if events1v.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_1v.csv"]
    urls = ["xgtd://data_1v/wls_day-{:02d}_1v.csv".format(_) for _ in range(2,91)]
    events1v.load(urls)
    print_data_summary()

NameError: name 'events1v' is not defined

In [8]:
%%time
if events2v.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_2v.csv"]
    urls = ["xgtd://wls_day-{:02d}_2v.csv".format(_) for _ in range(2,91)]
    events2v.load(urls)
    print_data_summary()

NameError: name 'events2v' is not defined

In [9]:
%%time
if netflow.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/nf_day-85.csv"]
    urls = ["xgtd://nf_day-{:02d}.csv".format(_) for _ in range(2,91)]
    netflow.load(urls)
    print_data_summary()

NameError: name 'netflow' is not defined

In [10]:
# Utility function to launch queries and show job number:
#   The job number may be useful if a long-running job needs
#   to be canceled.

def run_query(query, table_name = "answers", drop_answer_table=True, show_query=False):
    if drop_answer_table:
        conn.drop_frame(table_name)
    if query[-1] != '\n':
        query += '\n'
    query += 'INTO {}'.format(table_name)
    if show_query:
        print("Query:\n" + query)
    job = conn.schedule_job(query)
    print("Launched job {}".format(job.id))
    conn.wait_for_job(job)
    table = conn.get_table_frame(table_name)
    return table

In [11]:
# Generate a new edge frame for holding only the RDP edges
conn.drop_frame('RDPflow')
rdpflow = conn.create_edge_frame(
            name='RDPflow',
            schema=netflow.schema,
            source=devices,
            target=devices,
            source_key='srcDevice',
            target_key='dstDevice')
rdpflow

NameError: name 'conn' is not defined

In [ ]:
%%time
q = """
MATCH ()-[edge:Netflow]->()
WHERE edge.dstPort=3389
MERGE (v0: Devices { device : edge.srcDevice })
MERGE (v1: Devices { device : edge.dstDevice })
CREATE (v0)-[e:RDPflow {epochtime : edge.epochtime,
  duration : edge.duration, protocol : edge.protocol,
  srcPort : edge.srcPort, dstPort : edge.dstPort,
  srcPackets : edge.srcPackets, dstPackets : edge.dstPackets,
  srcBytes : edge.srcBytes, dstBytes : edge.dstBytes}]->(v1)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

In [ ]:
%%time
q = """
MATCH ()-[edge:Netflow]->()
WHERE edge.srcPort=3389
MERGE (v0: Devices { device : edge.srcDevice })
MERGE (v1: Devices { device : edge.dstDevice })
CREATE (v1)-[e:RDPflow {epochtime : edge.epochtime,
  duration : edge.duration, protocol : edge.protocol,
  srcPort : edge.dstPort, dstPort : edge.srcPort,
  srcPackets : edge.dstPackets, dstPackets : edge.srcPackets,
  srcBytes : edge.dstBytes, dstBytes : edge.srcBytes}]->(v0)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

In [ ]:
data=None
if rdpflow.num_edges == 0:
    print("RDPflow is empty")
elif rdpflow.num_edges <= 1000:
    data = rdpflow.get_data_pandas()
else:
    data = 'RDPflow (edges): {:,}'.format(rdpflow.num_edges)
data

In [ ]:
# Utility to print the data sizes currently in xGT
def print_netflow_data_summary():
  print_data_summary()
  print('RDPflow (edges): {:,}'.format(rdpflow.num_edges))

print_netflow_data_summary()

In [ ]:
# Generate an output edge frame for holding lateral movement edges
latmvmt_schema = netflow.schema
latmvmt_schema.append(['privEsc_epochtime', xgt.INT])
latmvmt_schema.append(['privEsc_processName', xgt.TEXT])
latmvmt_schema.append(['privEsc_parentProcessName', xgt.TEXT])
latmvmt_schema.append(['hijack_epochtime', xgt.INT])
latmvmt_schema.append(['hijack_processName', xgt.TEXT])
latmvmt_schema.append(['hijack_parentProcessName', xgt.TEXT])
latmvmt_schema.append(['edge_role_in_answer', xgt.TEXT])

conn.drop_frame('LatMvmt')
latmvmt = conn.create_edge_frame(
            name='LatMvmt',
            schema=latmvmt_schema,
            source=devices,
            target=devices,
            source_key='srcDevice',
            target_key='dstDevice')
latmvmt

In [ ]:
%%time
import string

thresholds = {
    'time_threshold_between_step' : 3600,   # one hour
    'time_threshold_hijack' : 180,          # three minutes
    'time_threshold_one_step' : 480,        # eight minutes
}
q = string.Template("""
MATCH (A)-[rdp1:RDPflow]->(B)-[rdp2:RDPflow]->(C),
      (A)-[hijack1:Events1v]->(A)-[privEsc1:Events1v]->(A),
      (B)-[hijack2:Events1v]->(B)-[privEsc2:Events1v]->(B)
WHERE A <> B AND B <> C AND A <> C 
  AND privEsc1.eventID = 4688 
  AND (privEsc1.processName = "Proc336322.exe" OR privEsc1.processName = "Proc695356.exe")
  AND hijack1.eventID = 4688 AND hijack1.processName = "Proc249569.exe"
  AND privEsc2.eventID = 4688 
  AND (privEsc2.processName = "Proc336322.exe" OR privEsc2.processName = "Proc695356.exe")
  AND hijack2.eventID = 4688 AND hijack2.processName = "Proc249569.exe"

  // Check time constraints on the overall pattern
  AND rdp1.epochtime <= rdp2.epochtime
  AND rdp2.epochtime - rdp1.epochtime < $time_threshold_between_step

  // Check time constraints on step from A to B
  AND privEsc1.epochtime <= hijack1.epochtime
  AND hijack1.epochtime <= rdp1.epochtime
  AND rdp1.epochtime - hijack1.epochtime < $time_threshold_hijack
  AND rdp1.epochtime - privEsc1.epochtime < $time_threshold_one_step

  // Check time constraints on step from B to C
  AND privEsc2.epochtime <= hijack2.epochtime
  AND hijack2.epochtime <= rdp2.epochtime
  AND rdp2.epochtime - hijack2.epochtime < $time_threshold_hijack
  AND rdp2.epochtime - privEsc2.epochtime < $time_threshold_one_step
MERGE (v0: Devices { device : rdp1.srcDevice })
MERGE (v1: Devices { device : rdp1.dstDevice })
MERGE (v2: Devices { device : rdp2.dstDevice })
CREATE (v0)-[e:LatMvmt { epochtime : rdp1.epochtime,
  duration : rdp1.duration, protocol : rdp1.protocol,
  srcPort : rdp1.srcPort, dstPort : rdp1.dstPort,
  srcPackets : rdp1.srcPackets, dstPackets : rdp1.dstPackets,
  srcBytes : rdp1.srcBytes, dstBytes : rdp1.dstBytes,
  privEsc_epochtime : privEsc1.epochtime,
  privEsc_processName : privEsc1.processName,
  privEsc_parentProcessName : privEsc1.parentProcessName,
  hijack_epochtime : hijack1.epochtime,
  hijack_processName : hijack1.processName,
  hijack_parentProcessName : hijack1.parentProcessName,
  edge_role_in_answer : "AB"}]->(v1)
CREATE (v1)-[f:LatMvmt { epochtime : rdp2.epochtime,
  duration : rdp2.duration, protocol : rdp2.protocol,
  srcPort : rdp2.srcPort, dstPort : rdp2.dstPort,
  srcPackets : rdp2.srcPackets, dstPackets : rdp2.dstPackets,
  srcBytes : rdp2.srcBytes, dstBytes : rdp2.dstBytes,
  privEsc_epochtime : privEsc2.epochtime,
  privEsc_processName : privEsc2.processName,
  privEsc_parentProcessName : privEsc2.parentProcessName,
  hijack_epochtime : hijack2.epochtime,
  hijack_processName : hijack2.processName,
  hijack_parentProcessName : hijack2.parentProcessName,
  edge_role_in_answer : "BC"}]->(v2)
RETURN COUNT(*)
""").substitute(thresholds)

answer_table = run_query(q)
print('Number of answers: {:,}'.format(answer_table.get_data()[0][0]))
print('Number of edges in output graph: {:,}'.format(latmvmt.num_rows))

In [ ]:
# retrieve the answer rows to the client in a pandas frame
data = latmvmt.get_data_pandas()
data[0:10]

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Ignore matplotlib warnings
import warnings
warnings.filterwarnings("ignore")

# http://jonathansoma.com/lede/algorithms-2017/classes/networks/networkx-graphs-from-source-target-dataframe/

In [ ]:
g = nx.from_pandas_edgelist(data, source='srcDevice', target='dstDevice') 

plt.figure(figsize=(12, 12))
layout = nx.spring_layout(g,iterations=50)
plt.axis('off')
plt.title("Lateral Movement threat detection with xGT on Superdome Flex")

# Plot it
nx.draw(g, with_labels=True)
plt.show()